In [1]:
# O código a seguir será a criação dos videos de metadados, o princípio para a criação dos dados a serem analisados

import cv2
from pynput import keyboard

# Variável global que marca se 'a' está sendo pressionado
a_pressionado = False

# Função chamada quando uma tecla é pressionada
def ao_pressionar(tecla):
    global a_pressionado
    try:
        if tecla.char == 'a':
            a_pressionado = True
    except AttributeError:
        pass

# Função chamada quando uma tecla é solta
def ao_soltada(tecla):
    global a_pressionado
    try:
        if tecla.char == 'a':
            a_pressionado = False
    except AttributeError:
        pass

# Inicia o listener do teclado
listener = keyboard.Listener(on_press=ao_pressionar, on_release=ao_soltada)
listener.start()

# Captura de vídeo
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = 20
codec = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(
    r"C:\Users\claudio.barradas\PycharmProjects\Domotic\teste\AssistiveDomotical\videos\felipe.mp4",
    codec, fps, (frame_width, frame_height)
)


print("Gravando... Segure 'a' para ponto verde. Pressione 'q' para sair.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Desenha o ponto com a cor baseada na variável global
    cor = (0, 255, 0) if a_pressionado else (0, 0, 255)
    cv2.circle(frame, (20, 20), 10, cor, -1)

    cv2.imshow('Webcam', frame)
    out.write(frame)

    # Sai com tecla 'q' detectada pelo OpenCV
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera tudo
cap.release()
out.release()
cv2.destroyAllWindows()
listener.stop()


ModuleNotFoundError: No module named 'pynput'

In [ ]:
# O seguinte código utiliza de um observador feito com o MediaPipe para a análise dos videos criados no passo anterior.
# Note que foi criado um coletor de dados que só recebe o que o observador envia e salva em um array para criar as tabelas no final.

from padroes import abcClasses
from observables.observableMediaPipe import ObservableMediaPipe

import time
import pandas as pd
import os

class ColetorDados(abcClasses.Observer):

    dados = []

    def mostrar(self, data):
        print("")
        print("#################")
        print("ear: ", data.ear)
        print("pontos olho:", data.pontos_olho)
        print("tempo: ", data.timestamp)
        print("frame: ", data.frame)
        print("#################")
        print("")

    def update(self, data):

        self.dados.append({
            "frame_idx": data.frame,
            "timestamp": data.timestamp,
            "ear": data.ear,
            "pontos_olho": data.pontos_olho,
            "fps": 1/ (time.time() - data.inicio),
            "tempo_processamento": time.time() - data.inicio,
            "piscando" : data.cor
        })

    def retorna_dados(self):
        return self.dados


In [ ]:
for video in os.listdir("AssistiveDomotical\\videos\\"):
    dadosMediaPipe = ColetorDados()

    caminho_video = "AssistiveDomotical\\videos\\" + video
    print(f"Processando {caminho_video}")


    print("\nProcesso MediaPipe:")

    olhoMediaPipe = ObservableMediaPipe(caminho_video)
    olhoMediaPipe.attach(dadosMediaPipe)
    olhoMediaPipe.execute()

    print("Processo MediaPipe Concluido!")
    print("Criando Tabela:")

    dfMediaPipe = pd.DataFrame(dadosMediaPipe.retorna_dados())
    dfMediaPipe.to_csv(f"AssistiveDomotical\\OSEMN\\obtain\\tabelasBrutas\\{video}MP.csv", index=False)